In [1]:
import torch
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler as ss
from sklearn.utils import shuffle

In [2]:
data = pd.read_csv("./iris.csv")
data = data.drop(["Id"], axis=1)

un_labels = data["Species"].unique()
# for i in data["Species"].unique():
#     print(sum(data["Species"]==i))

data = shuffle(data)
labels = data.pop("Species")
labels = np.array(labels)

data = ss().fit_transform(data)
print(un_labels)

['Iris-setosa' 'Iris-versicolor' 'Iris-virginica']


In [3]:
ohe_labels = np.zeros((len(labels), len(un_labels)))

for i in range(len(un_labels)):
    x = np.where(labels == un_labels[i])
    ohe_labels[x,i] = 1

ohe_labels[:15]

array([[1., 0., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [1., 0., 0.]])

In [4]:
features = torch.tensor(np.array(data), dtype=torch.float32)
labels = torch.tensor(ohe_labels, dtype=torch.float32)

In [8]:
print(data.shape,features.shape)

(150, 4) torch.Size([150, 4])


In [13]:
model = torch.nn.Sequential(
    torch.nn.Linear(4,7),
    torch.nn.Softmax(),
    torch.nn.Linear(7,4),
    torch.nn.Softmax(),
    torch.nn.Linear(4,3),
    torch.nn.Softmax()
)

preds = model(features)
print(preds[:5])

tensor([[0.3972, 0.3475, 0.2553],
        [0.3955, 0.3495, 0.2549],
        [0.3955, 0.3495, 0.2550],
        [0.3972, 0.3476, 0.2552],
        [0.3950, 0.3493, 0.2557]], grad_fn=<SliceBackward0>)


c:\Users\v.jahnavi\AppData\Local\Programs\Python\Python39\lib\site-packages\torch\nn\modules\container.py:139: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


In [14]:
loss_fn = torch.nn.BCELoss()

loss = loss_fn(preds, labels)
print(loss)

tensor(0.6434, grad_fn=<BinaryCrossEntropyBackward0>)


In [ ]:
optimize= torch.optim.SGD(model.parameters(),lr=0.1e-1)

In [ ]:
from torch import no_grad

epochs=100000
for epoch in range(epochs):
    preds=model(features)
    loss=loss_fn(preds,labels)
    loss.backward()
    optimize.step()
    optimize.zero_grad()
    with torch.no_grad():
        if (epoch+1)%(epochs//10)==0:
            right=0
            for i in range(len(preds)):
                for j in range(len(preds[i])):
                    
                    if preds[i][j]==max(preds[i]):
                        #print(preds[i][j])
                        #print(max(preds[i]),preds[i][j])
                        #print(labels[i])
                        if(labels[i][j]==1):
                            #print(labels[i])
                            right+=1
                            #print(right)
            print("Accuracy :",round(right*100/len(preds),5))

Accuracy : 98.66667
Accuracy : 98.66667
Accuracy : 98.66667
Accuracy : 98.66667
Accuracy : 98.66667
Accuracy : 98.66667
Accuracy : 98.66667
Accuracy : 98.66667
Accuracy : 98.66667
Accuracy : 98.66667
